# How to get the most out of this notebook
## Main points to understand first, Tl;Dr
* Don't worry about artificial distinctions or paradigms. Don't be afraid of new things.
* Instead, use specific sets of tools like type checkers, design patterns, and language features to your advantage.
* Use those tools judiciously to achieve your goals (like program efficiency, user/programmer experience, and correctness). 
* Correctness is _the single most important_ property of a program.

## Steps
* Open in VSCode with Pylance enabled. 
* Make sure your pyrightconfig.json has   "typeCheckingMode": "strict"
* Hover over code to get inferences from Pyright. 
  * This includes both inferred types and errors.
* Go through the cells in order, reading the markdown text and code in order. It's like a story book.

### Programming Concepts (Feel free to skip if you're in a hurry)
* There are no "paradigms". OO or functional programming are not categories or "paradigms" of languages, but instead (fuzzy) sets of PL _features_. 
  * E.g. is Python OO or FP? Well, it has classes, objects, and methods. It also has map and filter, as well as very Pythonic syntactic sugar for complex compositions of those higher-order functions (it's called a list comprehension).
* There is also no well-defined idea of "strong" or "weak" typing. These terms are used loosely to refer to certain sets of PL typing features. Usually, "strongly typed" means there are certain obligatory type annotations (see below).  
* Static typing vs. dynamic typing. 
  * It is NOT an either-or. It is NOT a spectrum. It is at least two independent dimensions. 
  * Case in point: C is statically but not dynamically typed. Java is both statically and dynamically typed. Python2 is dynamically but not statically typed. Assembly is neither.
  * Typing is a set of _features_ of a PL (programming language), not a type of PL.  
* Type _inference_ vs. type _annotation_.
  * Type inference is the process of outputting type information given source code. Since it is done only from source code and not from program execution, by definition the type information is _static_. This process is done in different ways by different static analysis tools like ghc, tsc, javac or Pyright. Essentially, this is where the _computer_ tells _you_ (and itself) the static type.
  * Type inference is not the same thing as manual type annotation or type hinting. Type annotation is the inclusion of explicit type information inside source code for use by both static analysis tools and human readers. Sometimes it is obligatory (e.g. Java) and sometimes not (e.g. Python3). Essentially, this is where _you_ tell the _computer_ the static type.
* Type safety and proof of correctness
  * Correctness is the most important property of a program. If it outputs the wrong answer (or crashes, or hangs forever) then it is useless no matter how efficiently it runs, how nice the user/programmer interface is, or how maintainable the code is. EXAMPLES:
    * Consider a web app for buying plane tickets. Imagine it takes 1 minute to get a confirmation. Now imagine it runs in 1 second, but you show up to the airport and the flight is actually sold out and they don't have your confirmation.
    * Some institutions (like banks) run extremely old Cobol programs that no one can reliably change anymore. It still works, and it's better than no working code.    
  * Type inference and annotation/hinting are each features of PLs that can be used in different ways by static analysis tools to reduce program errors.
    * For example, you can ask Pyright to infer all of your types with no explicit annotations. You can further constrain your program's types by adding annotations.
    * When considering program correctness, there is no important difference between an explicit, annotated type and an inferred type. They are different kinds of _constraints_ on the static types of a program.
  * Using a combination of constraints from type hints and inference procedures, tools like Pyright can statically detect errors in your program if they exist. 
    * This is also what compilers like javac (or gcc) do. If you get the types wrong, javac will fail and not output any Java bytecode. If it _does_ compile (or if Pyright reports no errors), then you are guaranteed a certain degree of correctness during program execution, depending on the exact PL and static analysis specifications. 
  * Notice that the goal of type safety is the same as the goal of test cases: proving correctness. The more you lean on your static analysis tools, the less you have to rely on tests. Put another way, static analysis will save you time to write _better_ tests that can prevent even more errors. 
    * Also notice that tests cannot run statically. You have to actually run some piece of your code to get a proof of correctness from a test case.
* Functions as data
  * Functions are already data no matter what PL you are working in. Your PL may be hiding this fact via its own higher-level abstractions. At the lowest level (i.e. in the von Neumann architecture), programs, which includes functions, are data.
  * Some PLs introduce high-level abstractions that maintain the treatment of functions as data. Many of these are known as "functional PLs", but that term is misleading just like "strongly typed". 
    * For example, like Haskell, Python and Java have high-level abstractions for functions as data. Namely, anonymous functions (lambda expressions).

# Part 0

Please install lm utils 0.0.21:

`pip install lastmile-utils==0.0.21 --force`

# Part 1: Basic error handling
Let's say we're building a local editor that allows you to load an AIConfig
from a local file and then run methods on it.

In the (simplified) code below, we do just that.

In [16]:
import json
from typing import Any


def read_json_from_file(path: str) -> dict[str, Any]:
    with open(path, "r") as f:
        return json.loads(f.read())
    

def start_app(path: str):
    """Load an AIConfig from a local path and do something with it."""
    aiconfig = read_json_from_file(path)
    print(f"Loaded AIConfig: {aiconfig['name']}\n")


start_app("cookbooks/Getting-Started/travel.aiconfig.json")

Loaded AIConfig: NYC Trip Planner




## Cool, LGTM, ship it!

# A few hours later...

# Issue #9000 Editor crashes on new file path
### opened 2 hours ago by lastmile-biggest-fan

Dear LastMile team,
I really like the editor, but when I give it a new file path, it crashes!
I was hoping it would create a new AIConfig for me and write it to the file...

# OK, what happened?

In [17]:
start_app("i-dont-exist-yet-please-create-me.json")

FileNotFoundError: [Errno 2] No such file or directory: 'i-dont-exist-yet-please-create-me.json'

# Oops

Ok, let's diagnose the problem here. We forgot to handle the case where the path doesn't exist.

That's understandable. As programmers, we don't always write perfect code.
Sometimes it's helpful to bring new tools into the workflow to prevent this kind of problem in the future.


Hmm, ok. Wouldn't it be nice if we had a static analyzer that could have caught this problem immediately? That way we could have fixed it before the initial PR was merged.

Let's analyze some tools.

## V2: Optional

First, let's fix the root cause and catch exceptions. Now, what do we do in the `except` block? 

Well, we can reraise, but that brings us right back to the previous case and doesn't achieve anything helpful. 

Instead, notice what happens if we return None and type hint the function accordingly (Optional[...]).

In [6]:
from typing import Any, Optional


def read_json_from_file(path: str) -> Optional[dict[str, Any]]:
    try:
        with open(path, "r") as f:
            return json.loads(f.read())
    except Exception as e:
        return None
    

def start_app(path: str):
    """Load an AIConfig from a local path and do something with it."""
    aiconfig = read_json_from_file(path)
    print(f"Loaded AIConfig: {aiconfig['name']}\n")

print("""
[Pyright] Object of type "None" is not subscriptable
PylancereportOptionalSubscript
(variable) aiconfig: dict[str, Any] | None
""")


[Pyright] Object of type "None" is not subscriptable
PylancereportOptionalSubscript
(variable) aiconfig: dict[str, Any] | None



# Aha!

Now, Pyright immediately tells us that `None` is a possibility, and we have to handle this case. Let's do that.


In [31]:
from typing import Optional
from aiconfig.Config import AIConfigRuntime



def read_json_from_file(path: str) -> Optional[dict[str, Any]]:
    try:
        with open(path, "r") as f:
            return json.loads(f.read())
    except Exception:
        return None

def start_app(path: str):
    """Load an AIConfig from a local path and do something with it."""
    aiconfig = read_json_from_file(path)
    if aiconfig is None:
        print(f"Could not load AIConfig from path: {path}. Creating and saving.")
        aiconfig = json.dumps(AIConfigRuntime.create())
        # [save the aiconfig to the path]        
        print(f"Loaded and saved new AIConfig\n")
    else:
        print(f"Loaded AIConfig: {aiconfig}\n")

start_app("cookbooks/Getting-Started/travel.aiconfig.json")
start_app("i-dont-exist-yet-please-create-me.json")

Loaded AIConfig: NYC Trip Planner

Loaded AIConfig: 



Ok, cool, much better. But wait, it would be nice to retain some information about what went wrong. My `None` value doesn't tell me anything about why the AIConfig couldn't be loaded. Does the file not exist? Was it a permission problem, networked filesystem problem? etc.

# V3: Result

The result library (https://github.com/rustedpy/result) provides a neat type
called `Result`, which is a bit like Optional. It's parametrized by the value type just like optional, but also by a second type for the error case.

We can use it like optional, but store an arbitrary value with information about what went wrong. Result also has more nice features we'll get to later.

In [2]:
from aiconfig.Config import AIConfigRuntime
from result import Result, Ok, Err
from typing import Any
import json
from json import JSONDecodeError


def read_json_from_file(path: str) -> Result[dict[str, Any], str]:    
    """
    The idea of this function is to quarantine the exceptions we are stuck with when using
    external code. We can't stop json.loads from raising, and we can't check it statically, 
    but we can immediately catch anything raised at the lower boundary of our application.
    This allows us to type check everything _above_ this function, enabling type-safe reuse.

    Specifically, we use a string in the error case to contain a helpful error message.
    """
    try:
        with open(path, "r") as f:
            return Ok(json.loads(f.read()))
    except FileNotFoundError:
        return Err(f"File not found at path: {path}")
    except OSError as e:
        return Err(f"Could not read file at path: {path}: {e}")
    except JSONDecodeError as e:
        return Err(f"Could not parse JSON at path: {path}: {e}")

def start_app(path: str):
    """Load an AIConfig from a local path and do something with it."""
    file_contents = read_json_from_file(path)
    match file_contents:
        case Ok(aiconfig_ok):
            print(f"Loaded AIConfig: {aiconfig_ok['name']}\n")
        case Err(e):
            print(f"Could not load AIConfig from path: {path} ({e}). Creating and saving.")
            aiconfig = AIConfigRuntime.create().model_dump(exclude="callback_manager")
            # [Save to file path]
            # aiconfig.save(path)
            print(f"Created and saved new AIConfig: {aiconfig['name']}\n")

start_app("cookbooks/Getting-Started/travel.aiconfig.json")
start_app("i-dont-exist-yet-please-create-me.json")

Loaded AIConfig: NYC Trip Planner

Could not load AIConfig from path: i-dont-exist-yet-please-create-me.json (File not found at path: i-dont-exist-yet-please-create-me.json). Creating and saving.
Created and saved new AIConfig: 



There are several nice things about this pattern:
* If you fail to check for the error case, you get static errors similar to the `None` Optional case
* You also get specific, useful error information unlike Optional
* Structural pattern matching: When matching the cases, you can elegantly and safely unbox the data inside the result.
* Because of pyright's ability to check for exhaustive pattern matching, it will yell at you if you don't handle the Err case. Try it! Comment out the Err case.

# Part 2: Composition

Cool, so we have a very basic example of better error handling. What about a more realistic level of complexity involving a sequence of chained operations? Consider this variant of the previous app example:

In [5]:
from dataclasses import dataclass
import json
from typing import NewType

JSON = NewType("JSON", dict[str, Any])

@dataclass
class HTTPResponse:
    status_code: int
    body: JSON

def read_file(path: str) -> str:
    with open(path, "r") as f:
        return f.read()
    

def parse_json(json_str: str) -> JSON:
    return json.loads(json_str)


def read_json_from_file(path: str) -> JSON:
    with open(path, "r") as f:
        return json.loads(f.read())

def get_prompt_text_list(aiconfig: JSON) -> list[str]:
    return [prompt["text"] for prompt in aiconfig["prompts"]]


def endpoint(path: str):
    """Load an AIConfig from a local path and do something with it."""
    contents = read_file(path)
    aiconfig = parse_json(contents)
    prompts = get_prompt_text_list(aiconfig)
    return HTTPResponse(200, JSON({"prompts": prompts}))



endpoint("cookbooks/Getting-Started/travel.aiconfig.json")

KeyError: 'text'

This is obviously bad. But it's not fatal, right? The server framework will handle this with catch-all exception handling. But that's not great either: the front-end can't do anything useful with a default error response.

It's better if we return specific codes as part of an explicit contract. Let's create an application protocol.

In [7]:
from dataclasses import dataclass
import json
from typing import NewType

JSON = NewType("JSON", dict[str, Any])

@dataclass(frozen=True)
class HTTPResponse:
    status_code: int
    body: JSON

def read_file(path: str) -> str:
    with open(path, "r") as f:
        return f.read()
    

def parse_json(json_str: str) -> JSON:
    return json.loads(json_str)


def get_prompt_text_list(aiconfig: JSON) -> list[str]:
    return [prompt["text"] for prompt in aiconfig["prompts"]]


def prompt_list_to_http_response(prompts: list[str]) -> HTTPResponse:
    return HTTPResponse(200, JSON({"prompts": prompts}))

def endpoint(path: str):
    """Load an AIConfig from a local path and do something with it."""

    try:
        contents = read_file(path)
        aiconfig = parse_json(contents)
        prompts = get_prompt_text_list(aiconfig)
        response = prompt_list_to_http_response(prompts)
        return response
    except FileNotFoundError:
        return HTTPResponse(404, JSON({"error": "File not found"}))
    except OSError as e:
        return HTTPResponse(502, JSON({"error": f"Could not read file: {e}"}))
    except json.JSONDecodeError as e:
        return HTTPResponse(555, JSON({"error": f"Could not parse JSON: {e}"}))
    except KeyError as e:
        return HTTPResponse(567, JSON({"error": f"Could not find key: {e}"}))


endpoint("cookbooks/Getting-Started/travel.aiconfig.json")

HTTPResponse(status_code=567, body={'error': "Could not find key: 'text'"})

This is better, but we can't reuse any of our helper functions! Every endpoint will have to repeat this error handling. 

Also, remember that exceptions are not statically checkable. Even if Pyright passes, we can accidentally raise (now or in the future) some other exception, and it will bubble up above `endpoint()` and break our protocol.

OK, new version:


In [24]:
from dataclasses import dataclass
from enum import Enum
import json
from typing import NewType
from result import Result, Ok, Err

JSON = NewType("JSON", dict[str, Any])

@dataclass(frozen=True)
class InternalFailure:
    """
    This type defines the server-client contract for server-side failures.
    It contains metadata about the failure, including a message, category, and the exception.
    Use the `to_json()` method to construct a corresponding HTTP response.
    """
    class Category(Enum):
        FileSystemError = "FileSystemError"
        ParseJSONError = "ParseJSONError"
        GetPromptError = "GetPromptError"

    message: str
    category: Category
    exception: Exception

    @staticmethod
    def from_exception(
        exception: Exception,
        category: Category     
    ) -> "InternalFailure":
        return InternalFailure(
            message=str(exception),
            category=category,
            exception=exception,
        )    

    def to_json(self) -> JSON:
        return JSON({
            "message": self.message,
            "category": self.category.value,
            "exception": str(self.exception),
        })

@dataclass(frozen=True)
class HTTPResponse:
    status_code: int
    body: JSON

def read_file(path: str) -> Result[str, InternalFailure]:
    try:
        with open(path, "r") as f:
            return Ok(f.read())
    except Exception as e:
        return Err(InternalFailure.from_exception(e, InternalFailure.Category.FileSystemError))
    

def parse_json(json_str: str) -> Result[JSON, InternalFailure]:
    try: 
        return Ok(json.loads(json_str))
    except json.JSONDecodeError as e:
        return Err(InternalFailure.from_exception(e, InternalFailure.Category.ParseJSONError))



def get_prompt_text_list(aiconfig: JSON) -> Result[list[str], InternalFailure]:
    try:
        return Ok([prompt["text"] for prompt in aiconfig["prompts"]])
    except KeyError as e:
        return Err(InternalFailure.from_exception(e, InternalFailure.Category.GetPromptError))


def prompt_list_to_http_response(prompts: list[str]) -> HTTPResponse:
    return HTTPResponse(200, JSON({"prompts": prompts}))


def internal_failure_to_http_response(failure: InternalFailure) -> HTTPResponse:
    """
    Statically guarantee exhaustive matching on InternalFailure.Type.
    Try commenting out one of the cases to see what happens!
    Try adding a new case that isn't one of the enum values to see what happens!
    """
    def _get_code(category: InternalFailure.Category) -> int:
        match category:
            case InternalFailure.Category.FileSystemError:
                return 502
            case InternalFailure.Category.ParseJSONError:
                return 555
            case InternalFailure.Category.GetPromptError:
                return 567
            
    code = _get_code(failure.category)
    return HTTPResponse(code, JSON({"response_type": "error", "data": failure.to_json()}))


def endpoint(path: str):
    """Load an AIConfig from a local path and do something with it."""

    file_contents = read_file(path)
    match file_contents:
        # Try removing one of the Err cases to see what happens!
        case Ok(file_contents_ok):
            parsed_json = parse_json(file_contents_ok)
            match parsed_json:
                case Ok(parsed_json_ok):
                    prompts = get_prompt_text_list(parsed_json_ok)
                    match prompts:
                        case Ok(prompts_ok):
                            return prompt_list_to_http_response(prompts_ok)
                        case Err(e):
                            return internal_failure_to_http_response(e)
                case Err(e):
                    return internal_failure_to_http_response(e)
        case Err(e):
            return internal_failure_to_http_response(e)


endpoint("cookbooks/Getting-Started/travel.aiconfig.json")

HTTPResponse(status_code=567, body={'response_type': 'error', 'data': {'message': "'text'", 'category': 'GetPromptError', 'exception': "'text'"}})

Cool, so we have an internal failure type that allows us to easily respect our client-server contract, our functions are type-safe and reusable, and our endpoints are guaranteed to be correct (i.e. respect the contract).

But now we have to do this annoying nested error checking! 

(Note that Optional would have the same problem, plus no way to distinguish between different errors. They all get converted to None.)

Luckily, Result has a really nice way to deal with this situation.

In [12]:
from dataclasses import dataclass
from enum import Enum
import json
from typing import NewType
from result import Result, Ok, Err

JSON = NewType("JSON", dict[str, Any])

@dataclass(frozen=True)
class InternalFailure:
    """
    This type defines the server-client contract for server-side failures.
    It contains metadata about the failure, including a message, category, and the exception.
    Use the `to_json()` method to construct a corresponding HTTP response.
    """    
    class Category(Enum):
        FileSystemError = "FileSystemError"
        ParseJSONError = "ParseJSONError"
        GetPromptError = "GetPromptError"

    message: str
    category: Category
    exception: Exception

    @staticmethod
    def from_exception(
        exception: Exception,
        category: Category     
    ) -> "InternalFailure":
        return InternalFailure(
            message=str(exception),
            category=category,
            exception=exception,
        )    

    def to_json(self) -> JSON:
        return JSON({
            "message": self.message,
            "category": self.category.value,
            "exception": str(self.exception),
        })

@dataclass(frozen=True)
class HTTPResponse:
    status_code: int
    body: JSON
    

def read_file(path: str) -> Result[str, InternalFailure]:
    try:
        with open(path, "r") as f:
            return Ok(f.read())
    except Exception as e:
        return Err(InternalFailure.from_exception(e, InternalFailure.Category.FileSystemError))
    

def parse_json(json_str: str) -> Result[JSON, InternalFailure]:
    try: 
        return Ok(json.loads(json_str))
    except json.JSONDecodeError as e:
        return Err(InternalFailure.from_exception(e, InternalFailure.Category.ParseJSONError))


def get_prompt_text_list(aiconfig: JSON) -> Result[list[str], InternalFailure]:
    try:
        return Ok([prompt["text"] for prompt in aiconfig["prompts"]])
    except KeyError as e:
        return Err(InternalFailure.from_exception(e, InternalFailure.Category.GetPromptError))


def prompt_list_to_http_response(prompts: list[str]) -> HTTPResponse:
    return HTTPResponse(200, JSON({"prompts": prompts}))


def internal_failure_to_http_response(failure: InternalFailure) -> HTTPResponse:
    """
    Statically guarantee exhaustive matching on InternalFailure.Type.
    Try commenting out one of the cases to see what happens!
    Try adding a new case that isn't one of the enum values to see what happens!
    """
    def _get_code(category: InternalFailure.Category) -> int:
        match category:
            case InternalFailure.Category.FileSystemError:
                return 502
            case InternalFailure.Category.ParseJSONError:
                return 555
            case InternalFailure.Category.GetPromptError:
                return 567
            
    code = _get_code(failure.category)
    return HTTPResponse(code, JSON({"response_type": "error", "data": failure.to_json()}))


def endpoint(path: str):
    """Load an AIConfig from a local path and do something with it."""

    response = (
        read_file(path)
        .and_then(parse_json)
        .and_then(get_prompt_text_list)
        .map(prompt_list_to_http_response)
        .unwrap_or_else(internal_failure_to_http_response)
    )
    return response


endpoint("cookbooks/Getting-Started/travel.aiconfig.json")

HTTPResponse(status_code=567, body={'error': "Could not find key: 'text'"})

Neat. To recap, this code now has the following very nice properties:
* Unlike exceptions, errors are statically checked, eliminating a whole class of bugs.
* Modular and highly reusable in a type-safe way.
* Error information is retained, unlike Optional
* Concise syntax for chaining operations that can fail.

# Part 3: Advanced topics
* Higher-order functions
* do-notation for complex Result composition

## Higher-order functions
As our application grows, we will inevitably add new external dependencies which can raise exceptions. Let's ease the conversion of those exceptions
into statically-checkable Err values by abstracting out try-except. 

We can do this by leveraging the `core_utils.exception_handled` parametrized decorator. Note that a decorator is just a higher-order function,
specifically a (Function -> Function). It is a function that takes a function and returns a function. 
Python has a neat "@" syntax that is often used to apply a decorator.

Here we use a slight generalization of a decorator that I call a parametrized decorator. 

A decorator may only take one input, which is a function. In contrast, a parametrized decorator also takes more inputs which control the decoration at the point where the decorator is used.

In this case, it allows us to safely and concisely associate each "dangerous" function with specific metadata for internal error tracking. 

See the new code below:

In [4]:
from dataclasses import dataclass
from enum import Enum
from functools import partial
import json
from typing import Any, NewType, ParamSpec, TypeVar
from result import Err

import lastmile_utils.lib.core.api as core_utils
P = ParamSpec("P")
T = TypeVar("T")

JSON = NewType("JSON", dict[str, Any])

@dataclass(frozen=True)
class InternalFailure:
    """
    This type defines the server-client contract for server-side failures.
    It contains metadata about the failure, including a message, category, and the exception.
    Use the `to_json()` method to construct a corresponding HTTP response.
    """    
    class Category(Enum):
        FileSystemError = "FileSystemError"
        ParseJSONError = "ParseJSONError"
        GetPromptError = "GetPromptError"

    message: str
    category: Category
    exception: Exception

    @staticmethod
    def from_exception(
        exception: Exception,
        category: Category     
    ) -> "InternalFailure":
        return InternalFailure(
            message=str(exception),
            category=category,
            exception=exception,
        )    

    def to_json(self) -> JSON:
        return JSON({
            "message": self.message,
            "category": self.category.value,
            "exception": str(self.exception),
        })
    

@dataclass(frozen=True)
class HTTPResponse:
    status_code: int
    body: JSON


def _handle_exception(exception: Exception, category: InternalFailure.Category):
    """
    This is a straight-forward helper function. 
    Convert the exception and user-specified category into a Result[T, InternalFailure]
    """
    return Err(InternalFailure.from_exception(exception, category))


def convert_exceptions(category: InternalFailure.Category):
    """
    This is a higher-order function that returns a parametrized decorator.
    The code is extremely short, but powerful.

    Here's the idea: we want to output a parametrized decorator that can be used
    to convert exceptions to specific InternalFailure instances (with specific categories).

    We can leverage core_utils.exception_handled to do the core work, namely actually running try...except for us.

    `core_utils.exception_handled` is a parametrized decorator: it allows you to give an extra argument,
    the exception handler (Exception -> Result).

    In our case, we don't want to give an exception handler directly at every decorated function definition.
    This function wraps `core_utils.exception_handled` and returns a new parametrized decorator, which accepts just 
    the argument we care about: the value of InternalFailure.Category that we want to associate with a decorated function.

    """

    # This is where we create the handler that conforms to the `exception_handled` signature.
    # Specifically, exception_handled expects a function that takes one argument, an Exception.
    # Our function _handle_exception takes an Exception, but also a second argument, category.
    # Partial is exactly the tool to convert a function that takes some arguments into another function
    # that takes fewer arguments. 
    # In this case, we are binding the category passed into this function (convert_exceptions(category)) 
    # to the category argument of _handle_exception.
    # That creates a new function that just takes one argument, Exception, 
    # which is exactly what we need to pass into exception_handled!
    handler = partial(_handle_exception, category=category)

    # Here's another way to do this, but it doesn't play as well with type checking.
    # `handler = lambda e: _handle_exception(e, category)``

    # Now that we have created a handler that conforms to the `exception_handled` signature, we can pass it into
    # `exception_handled` and get a new decorator.
    # Specifically, we return a new parametrized decorator that takes a category as its extra input.
    return core_utils.exception_handled(handler)


"""
The decorated functions below use our decorator above to handle exceptions 
and convert them into InternalFailure instances with specific categories.

The decorator changes the function signature, which you can see as follows:

```
# Type `read_file on a new line and over over it. This is the output of the decorated `read_file`.
You'll see the inferred type is different from the signature of the nondecorated function.
read_file

# on-hover. Note that this `Ok | Err` union is nothing but a Result[str, InternalFailure].
(function) def read_file(path: str) -> (Ok[str] | Err[InternalFailure])
```
"""
@convert_exceptions(InternalFailure.Category.FileSystemError)
def read_file(path: str) -> str:
    with open(path, "r") as f:
        return f.read()
    

@convert_exceptions(InternalFailure.Category.ParseJSONError)
def parse_json(json_str: str) -> JSON:
    return JSON(json.loads(json_str))


@convert_exceptions(InternalFailure.Category.GetPromptError)
def get_prompt_text_list(aiconfig: JSON) -> list[str]:
    return [prompt["text"] for prompt in aiconfig["prompts"]]


def prompt_list_to_http_response(prompts: list[str]) -> HTTPResponse:
    return HTTPResponse(200, JSON({"prompts": prompts}))


def internal_failure_to_http_response(failure: InternalFailure) -> HTTPResponse:
    """
    Statically guarantee exhaustive matching on InternalFailure.Type.
    Try commenting out one of the cases to see what happens!
    Try adding a new case that isn't one of the enum values to see what happens!
    """
    def _get_code(category: InternalFailure.Category) -> int:
        match category:
            case InternalFailure.Category.FileSystemError:
                return 502
            case InternalFailure.Category.ParseJSONError:
                return 555
            case InternalFailure.Category.GetPromptError:
                return 567
            
    code = _get_code(failure.category)
    return HTTPResponse(code, JSON({"response_type": "error", "data": failure.to_json()}))


def endpoint(path: str):
    """Load an AIConfig from a local path and do something with it."""

    response = (
        read_file(path)
        .and_then(parse_json)
        .and_then(get_prompt_text_list)
        .map(prompt_list_to_http_response)
        .unwrap_or_else(internal_failure_to_http_response)
    )
    return response

endpoint("cookbooks/Getting-Started/travel.aiconfig.json")

HTTPResponse(status_code=567, body={'response_type': 'error', 'data': {'message': "'text'", 'category': 'GetPromptError', 'exception': "'text'"}})

Notice how powerful this program is per SLOC. It is provably free of unhandled exceptions except 
inside the tiny wrapper functions that call across the boundaries of our code into 3rd party libraries.

All of our logic is separated into small and reusable components.

As we work, Pyright gives immediate feedback about large classes of potential errors.
We can work with Pyright and Copilot to quickly write obviously- and provably-correct code just based on making the static types work.

Notice a few things we did not need to use or worry about at all:
* Inheritance or class-based polymorphism (we implicitly used parametric polymorphism when calling `exception_handled`, but I bet you didn't notice or care.)
* State mutation or "encapsulation"
* Special cases
* Unconstrained types like `Any`, untyped `*args/**kwargs`, or incomplete generic types like `list`.
* `None`, `Optional`. We eliminated the Python equivalent of `NullpointerException`! 
* `try...except`, `raise`, or re-`raise`
* Any of a large body of design patterns like Singleton, Factory, or Proxy



It's just data and functions, that's it. And as we noted earlier, functions are data, so really it's just data.

## Do-notation

Anyway, cool. Now let's add a requirement to our server to make it a little more realistic.
Let's suppose that we want to allow the endpoint caller to read from a local CSV, extract a text filter, 
and apply that filter inside `get_prompt_text_list()`.

We make a few small, straight-forward modifications to the code above, but run into a problem
inside `endpoint()`.

We still want to do `.and_then(get_prompt_text_list)` but that no longer type checks, because `and_then()` only accepts a function with one argument. Since `get_prompt_text_list` now takes another argument, this doesn't work.

(Actually, we run into a similar problem inside `get_prompt_filter()` but in that case we have an easy fix using `partial`. See if you can figure out why that won't work for `get_prompt_text_list()`.)


Skip down to the new `endpoint()` to see what's going on:



In [9]:
from dataclasses import dataclass
from enum import Enum
from functools import partial
import csv
from io import StringIO

import json
from typing import Any, NewType, ParamSpec, TypeVar
from result import Err

import lastmile_utils.lib.core.api as core_utils
P = ParamSpec("P")
T = TypeVar("T")

JSON = NewType("JSON", dict[str, Any])


@dataclass(frozen=True)
class InternalFailure:
    """
    This type defines the server-client contract for server-side failures.
    It contains metadata about the failure, including a message, category, and the exception.
    Use the `to_json()` method to construct a corresponding HTTP response.
    """    
    class Category(Enum):
        FileSystemError = "FileSystemError"
        ParseJSONError = "ParseJSONError"
        GetPromptError = "GetPromptError"
        ParseCSVError = "ParseCSVError"

    message: str
    category: Category
    exception: Exception

    @staticmethod
    def from_exception(
        exception: Exception,
        category: Category     
    ) -> "InternalFailure":
        return InternalFailure(
            message=str(exception),
            category=category,
            exception=exception,
        )    

    def to_json(self) -> JSON:
        return JSON({
            "message": self.message,
            "category": self.category.value,
            "exception": str(self.exception),
        })
    

@dataclass(frozen=True)
class HTTPResponse:
    status_code: int
    body: JSON


def _handle_exception(exception: Exception, category: InternalFailure.Category):
    return Err(InternalFailure.from_exception(exception, category))


def convert_exceptions(category: InternalFailure.Category):   
    handler = partial(_handle_exception, category=category)
    return core_utils.exception_handled(handler)


@convert_exceptions(InternalFailure.Category.FileSystemError)
def read_file(path: str) -> str:
    with open(path, "r") as f:
        return f.read()
    

@convert_exceptions(InternalFailure.Category.ParseJSONError)
def parse_json(json_str: str) -> JSON:
    return JSON(json.loads(json_str))


@convert_exceptions(InternalFailure.Category.GetPromptError)
def lookup_filter(filter_key: str, filter_mapping: dict[str, str]) -> str:
    return filter_mapping[filter_key]


@convert_exceptions(InternalFailure.Category.GetPromptError)
def get_prompt_text_list(aiconfig: JSON, prompt_filter: str) -> list[str]:
    return [prompt["text"] for prompt in aiconfig["prompts"] if prompt_filter in prompt["text"]]


@convert_exceptions(InternalFailure.Category.ParseCSVError)
def parse_csv_to_mapping(csv_contents: str) -> dict[str, str]:
    """Input: like "key1,value1\nkey2,value2\nkey3,value3\n"""
    f = StringIO(csv_contents)
    reader = csv.reader(f, delimiter=',')
    return {row[0]: row[1] for row in reader}


def prompt_list_to_http_response(prompts: list[str]) -> HTTPResponse:
    return HTTPResponse(200, JSON({"prompts": prompts}))


def internal_failure_to_http_response(failure: InternalFailure) -> HTTPResponse:
    """
    Statically guarantee exhaustive matching on InternalFailure.Type.
    Try commenting out one of the cases to see what happens!
    Try adding a new case that isn't one of the enum values to see what happens!
    """
    def _get_code(category: InternalFailure.Category) -> int:
        match category:
            case InternalFailure.Category.FileSystemError:
                return 502
            case InternalFailure.Category.ParseJSONError:
                return 555
            case InternalFailure.Category.GetPromptError:
                return 567
            case InternalFailure.Category.ParseCSVError:
                return 593
            
    code = _get_code(failure.category)
    return HTTPResponse(code, JSON({"response_type": "error", "data": failure.to_json()}))


def get_prompt_filter(csv_path: str, filter_key: str) -> Result[str, InternalFailure]:
    return (
        read_file(csv_path)
        .and_then(parse_csv_to_mapping)
        .and_then(partial(lookup_filter, filter_key=filter_key))
    )

def endpoint(path: str, filter_key: str):
    # TODO: finish implementing this function.
    """Load an AIConfig from a local path and do something with it."""

    CSV_PATH = "my_filters.csv"

    prompt_filter = get_prompt_filter(CSV_PATH, filter_key)
    response = (
        read_file(path)
        .and_then(parse_json)
        # .and_then... what?

    )
    return response

# TODO: call this.
# endpoint("cookbooks/Getting-Started/travel.aiconfig.json", "some_key")

### Detour: List comprehensions

Python has an interesting and fundamental syntactical construct called a comprehension. Let's look at how this works with the list type.


In [16]:
my_list = [1, 2, 3, 4]
my_new_list = [x ** 2 for x in my_list if x % 2 == 0]
print(my_new_list)

[4, 16]


Ok, that's interesting. In this concise expression, are we doing a complex operation that can be described as follows:
* Construct a new list with values of the form `x ** 2` where `x` will take on different values.
* Bind `x` to the values in `my_list`, in sequence (preserving order), and use each of those values to construct the new list.
* Only include even values of `x`, values where `x % 2 == 0`.
* The output list is therefore a new list `[4, 16] == [2 ** 2, 4 ** 2]`.

This can be expressed in more traditional functional terms using higher-order functions. 

This code is equivalent:

In [18]:
my_list = [1, 2, 3, 4]
my_new_list = list(
    filter(
        lambda x: x % 2 == 0,
        map(lambda x: x ** 2, my_list)
    )
)

print(my_new_list)

[4, 16]


Interesting. Let's put that aside and consider a new function. This also uses a list comprehension, but in this cases we need two `for` expressions to flatten the input list.

In [11]:
def flatten(my_list: list[list[int]]) -> list[int]:
    return [
        item 
        for sublist in my_list 
        for item in sublist
    ]

input_list = [
    [1, 2, 3],
    [4, 5, 6], 
    [7]
]

print(flatten(input_list))

[1, 2, 3, 4, 5, 6, 7]


This syntax quickly becomes pretty perplexing, but it is fundamental to Python - it's been around since at least Python 2 (https://docs.python.org/2/tutorial/datastructures.html#list-comprehensions).

Here is the correct way to read the implementation of `flatten()`:
- Start at the first `for`. Unpack all the elements of `my_list` and assign each one in turn to `sublist`.
- Now proceed to the second `for`. Unpack all the elements of `sublist` and assign each one in turn to `item`.
- Now we are done evaluating the `for`s, so we go back to the first line of the comprehension.
- Construct your new list using each value of `item` in the order in which they were unpacked.

### Back to the problem at hand

Keep that double list comprehension thing in mind.


Now remember our problem with `and_then()`. Luckily, Result has a very handy solution to this, and it reads sort of like a list comprehension. 
You can think of it kind of like a Result comprehension. 

To understand this example well, make sure to hover over the identifiers and look at their inferred types, especially inside `endpoint()`.

In [8]:
from dataclasses import dataclass
from enum import Enum
from functools import partial
import csv
from io import StringIO

import json
from typing import Any, NewType, ParamSpec, TypeVar
from result import Err

import lastmile_utils.lib.core.api as core_utils
import result
P = ParamSpec("P")
T = TypeVar("T")

JSON = NewType("JSON", dict[str, Any])


@dataclass(frozen=True)
class InternalFailure:
    """
    This type defines the server-client contract for server-side failures.
    It contains metadata about the failure, including a message, category, and the exception.
    Use the `to_json()` method to construct a corresponding HTTP response.
    """    
    class Category(Enum):
        FileSystemError = "FileSystemError"
        ParseJSONError = "ParseJSONError"
        GetPromptError = "GetPromptError"
        ParseCSVError = "ParseCSVError"

    message: str
    category: Category
    exception: Exception

    @staticmethod
    def from_exception(
        exception: Exception,
        category: Category     
    ) -> "InternalFailure":
        return InternalFailure(
            message=str(exception),
            category=category,
            exception=exception,
        )    

    def to_json(self) -> JSON:
        return JSON({
            "message": self.message,
            "category": self.category.value,
            "exception": str(self.exception),
        })
    

@dataclass(frozen=True)
class HTTPResponse:
    status_code: int
    body: JSON


def _handle_exception(exception: Exception, category: InternalFailure.Category):
    return Err(InternalFailure.from_exception(exception, category))


def convert_exceptions(category: InternalFailure.Category):   
    handler = partial(_handle_exception, category=category)
    return core_utils.exception_handled(handler)


@convert_exceptions(InternalFailure.Category.FileSystemError)
def read_file(path: str) -> str:
    with open(path, "r") as f:
        return f.read()
    

@convert_exceptions(InternalFailure.Category.ParseJSONError)
def parse_json(json_str: str) -> JSON:
    return JSON(json.loads(json_str))


@convert_exceptions(InternalFailure.Category.GetPromptError)
def lookup_filter(filter_key: str, filter_mapping: dict[str, str]) -> str:
    return filter_mapping[filter_key]


@convert_exceptions(InternalFailure.Category.GetPromptError)
def get_prompt_text_list(aiconfig: JSON, prompt_filter: str) -> list[str]:
    return [prompt["text"] for prompt in aiconfig["prompts"] if prompt_filter in prompt["text"]]


@convert_exceptions(InternalFailure.Category.ParseCSVError)
def parse_csv_to_mapping(csv_contents: str) -> dict[str, str]:
    """Input: like "key1,value1\nkey2,value2\nkey3,value3\n"""
    f = StringIO(csv_contents)
    reader = csv.reader(f, delimiter=',')
    return {row[0]: row[1] for row in reader}


def prompt_list_to_http_response(prompts: list[str]) -> HTTPResponse:
    return HTTPResponse(200, JSON({"prompts": prompts}))


def internal_failure_to_http_response(failure: InternalFailure) -> HTTPResponse:
    """
    Statically guarantee exhaustive matching on InternalFailure.Type.
    Try commenting out one of the cases to see what happens!
    Try adding a new case that isn't one of the enum values to see what happens!
    """
    def _get_code(category: InternalFailure.Category) -> int:
        match category:
            case InternalFailure.Category.FileSystemError:
                return 502
            case InternalFailure.Category.ParseJSONError:
                return 555
            case InternalFailure.Category.GetPromptError:
                return 567
            case InternalFailure.Category.ParseCSVError:
                return 593
            
    code = _get_code(failure.category)
    return HTTPResponse(code, JSON({"response_type": "error", "data": failure.to_json()}))


def get_prompt_filter(csv_path: str, filter_key: str) -> Result[str, InternalFailure]:
    return (
        read_file(csv_path)
        .and_then(parse_csv_to_mapping)
        .and_then(partial(lookup_filter, filter_key=filter_key))
    )

def endpoint(path: str, filter_key: str):
    """Load an AIConfig from a local path and do something with it."""

    CSV_PATH = "my_filters.csv"
    
    # Much like a list comprehension, the correct way to read this is:
    # - Start at the first `for`. Run get_prompt_filter(), check if it returns an Ok,
    #   and if so, unpack the value and assign it to prompt_filter_ok.
    # - If it's an Err, short-circuit and the whole `do() evaluates to that Err.
    # - If it's Ok, continue to the next `for`.
    # - Now run read_file(), check if it returns an Ok, 
    #   and if so, unpack the value and assign it to aiconfig_path_contents_ok.
    # - If it's an Err, short-circuit and the whole `do() evaluates to that Err.
    # - If it's Ok, continue to the next `for`.
    # - Now run parse_json(), check if it returns an Ok,
    #   and if so, unpack the value and assign it to aiconfig_ok.
    # - If it's an Err, short-circuit and the whole `do() evaluates to that Err.
    # - If it's an Ok, then everything we ran returned Ok values, and 
    #   we are done evaluating the `for` expressions.
    #   Proceed to the first line of the `do()` block, which is what 
    #   the entire `do()` will evaluate to.
    #   Take our aiconfig_ok and prompt_filter_ok values, 
    #   run get_prompt_text_list() on them, and get the return value. 
    #   Now the `do()` block is done and evaluates to that return value, 
    #   which is a Result[list[str], InternalFailure].
    prompt_list = result.do(
        get_prompt_text_list(aiconfig_ok, prompt_filter_ok)
        for prompt_filter_ok in get_prompt_filter(CSV_PATH, filter_key)
        for aiconfig_path_contents_ok in read_file(path)
        for aiconfig_ok in parse_json(aiconfig_path_contents_ok)
    )

    response = (
        prompt_list
        .map(prompt_list_to_http_response)
        .unwrap_or_else(internal_failure_to_http_response)
    )

    return response

endpoint("cookbooks/Getting-Started/travel.aiconfig.json", "some_key")

HTTPResponse(status_code=502, body={'response_type': 'error', 'data': {'message': "[Errno 2] No such file or directory: 'my_filters.csv'", 'category': 'FileSystemError', 'exception': "[Errno 2] No such file or directory: 'my_filters.csv'"}})

Let's analyze the similarities and differences between `result.do()` and the double-for list comprehension used in `flatten()`. 

The two expressions have the same form, in the sense that they construct values of a specific container type by evaluating a sequence of `for` expressions. In both cases, you start at the first `for`, go in order until you're done with the last `for`, then come back up to the top and evaluate an expression.

Interesting.

Now, you might be wondering why they are so different despite those similarities. In `result.do()`, we have `Ok` and `Err` values, `Err` values short-circuit, and there's no actual iteration over a sequence. Every `Result` is exactly one `Ok` or exactly one `Err`. With list comprehensions, we are iterating over an arbitrary number of elements.

The answer is that this notation abstracts over a class of different concrete "chainable" types. Result is one such type; list is another. They are different by construction, and _defined_ by their distinct chaining rules. 

In the case of `list`, "chaining" is actually _defined_ to involve flattening. Actually, list chaining involves a little more than just flatten: it maps a function over each element, and then flattens. It's called `flatMap`:

In [23]:
from typing import Callable

T = TypeVar("T")
U = TypeVar("U")

# If Python had a flat_map function like the equivalents in 
# Javascript, Scala, or other languages, it would work like this.
def flatMap(my_list: list[T], f: Callable[[T], list[U]]) -> list[U]:
    return [
        output_item
        for input_item in my_list
        for output_item in f(input_item)
    ]

my_list = [1, 2, 3, 4]
my_new_list = flatMap(my_list, lambda x: [x, x * 2])

print(my_new_list)


# It could also be implemented like this:
def flat_map_v2(my_list: list[int], f: Callable[[int], list[int]]) -> list[int]:
    return flatten(list(map(f, my_list)))



[1, 2, 2, 4, 3, 6, 4, 8]


Now hopefully you can see the generality of `list` comprehensions, which is like the do-notation of the `list` type. Notice how easy it was to implement both `flatten` and `flatMap` (not to mention `filter`) using list comprehensions.

### Summary

So, Result is just another container type that happens to be chainable. There is `dict` (another container type which happens not to be chainable in any obvious way), there is `list`, and there is `Result.`

The essence of chaining list operations is iteration over the elements, applying list-producting function, and flattening. 
The essence of chaining Result operations is either short-circuiting an Err, or applying a Result-producing function to the `Ok` and then "flattening" the nested `Ok`.

Exercises:

* A natural question would be, "what is the `flatMap` of `Result`?" See if you can figure it out.
* Look at how we used `.map()` in our Result chain. `Result.map` applies a function inside a Result if it's an Ok, otherwise it just leaves the Err alone. What is the equivalent operation for lists? 

### Further reading
* Javascript Array flatMap: https://developer.mozilla.org/en-US/docs/Web/JavaScript/Reference/Global_Objects/Array/flatMap
* "Async/await just is the do-notation of the Promise Monad": https://gist.github.com/peter-leonov/c86720d1517235a1f28cd453a9d39bb4
* Returns library: https://returns.readthedocs.io/en/latest/
  * Although more featureful and instructive to look at, I prefer the Result library for its known Pyright compatibility.
* Haskell monad typeclass: https://wiki.haskell.org/Monad  
* Programming and Programming Languages (PAPL), Brown CS: https://papl.cs.brown.edu/2020/
* "Programming Languages:
Application and Interpretation": https://www.plai.org/3/2/PLAI%20Version%203.2.2%20electronic.pdf
* F# for Fun and Profit: https://fsharpforfunandprofit.com/

# Part 4: Methods are just functions

Did you notice something funny in the previous section? There's an unexplained syntactic difference between `Result`'s `map` and `list`'s `map`, even though they are doing the same kind of operation over their respective data structures.

This really is just a (superficial) syntactic difference. If you look at `Result`'s source code, you'll see it uses classes and methods. Despite this, `Result` is closely analogous to Haskell's `Either` type even though Haskell has no classes or objects in the sense that Python does.

What gives? How is this FP type implemented using OOP structures? Well, neither type of programming is a paradigm. They're just collections of PL features.

Funnily enough, Python's method syntax clarifies this.

In [26]:
# A snippet from Result's code: the implementation of Ok.and_then()`
def and_then(self, op: Callable[[T], Result[U, E]]) -> Result[U, E]:
    """
    The contained result is `Ok`, so return the result of `op` with the
    original value passed in
    """
    return op(self._value)

A method is literally nothing but a function whose first argument is the object containing the method. In Python, this is explicit and obligatory. 
 
Similarly, the syntax for calling a method is precisely isomorphic to a function call: swap the function and first argument and separate with a `.`.

A consequence of this fact is that an object is nothing but a struct bundled with a set of functions whose first argument are that object's type. 

Stripping away these superficial differences reveals the substantive differences between FP and OOP styles. Each style (or "paradigm" if you insist) can be analyzed into not much more than its distinct set of features. 

The most instructive way to understand the difference is not just "objects vs. functions". After all, objects have functions (they're called "methods") and FP languages have objects (they're called "structs", "records", etc.)

The main substantive differences are:
- The use of _nested function types_, namely higher-order functions in FP, vs. the lack of this in OOP.
- Referential transparency in FP. What you see is what you get. The output of a function only depends on that function and its inputs. This is also called purity, and is distinct from a program whose execution involves mutation or other side effects. One of the essential features of OOP is mutation, which is called "encapsulation" in OOP. In contrast, one of the essential features of FP is immutability or purity, which implies referential transparency.
- Code organization: OOP tends to organize around bundles of mutable state and functions that do that mutation, namely classes/objects. FP sometimes bundles related functions into modules, but does not organize around mutating a piece of state using those functions. 
- Different kinds of polymorphism: OOP-compatible languages like Python or Java bundle together code reuse (inheritance) and ad-hoc polymorphism (subtyping) through subclassing. FP often achieves the same objectives (at a high level) through higher-order functions, but there aren't obvious 1:1 feature mappings.
- One thing that is shared between OOP and FP is the other major type of polymorphism, parametric polymorphism, which is essentially generic types. This is distinct from ad-hoc polymorphism, which takes different forms in different languages. In OOP style, ad-hoc polymorphism usually comes in the form of dynamic dispatch via subclassing. Both FP and OOP styles use ad-hoc polymorphism in the form of overloading.